# EEG Batch Import

This notebook batch imports all the pairs of `xdf` and `json` files in the Data folder. Note that one must first create the `json` files by running the [eeg_processing](eeg_preprocessing.ipynb) notebook first.

## Import libraries

In [5]:
# Standard libraries
import mne
import json
from mne.preprocessing import ICA

# Custom libraries
from Functions import import_data


## Select files to import

Add the name of the files to import. Note that each file in the list must have a `xdf` and `json` file with that name.

In [6]:
# Settings
files_to_import = [
    "sub-P004_ses-S001_task-T1_run-001_eeg",
]

## Import and preprocess data



In [10]:
# Preallocate variables 
eeg_mne = [None] * len(files_to_import) # List of MNE objects with raw data

for file in files_to_import:
    # Import settings saved in json
    with open(f"Data\{file}.json", "r") as file_object:
        settings = json.load(file_object)

    # Import EEG data
    [eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(
        f"Data\{file}.xdf",
        picks=settings["ch_names"]
        )
    
    # Create MNE array
    info = mne.create_info(settings["ch_names"], eeg_fs, ch_types='eeg')    # Create info properties
    mne_raw = mne.io.RawArray(eeg_data, info=info)                          # Create MNE raw array
    mne_raw.set_montage('standard_1020')                                    # Set montage to array

    # Remove bad channels
    mne_raw.info["bads"] = settings["bad_chans"]
    mne_raw_rm_chans = mne_raw.copy().pick_types(eeg=True, exclude='bads')

    # Apply HPF
    filt_raw = mne_raw_rm_chans.copy().filter(
        l_freq=settings["hpf_fc"],
        h_freq=None,
        picks=settings["ch_names"]
        )
    
    # Create ICA
    ica = ICA(
        n_components = settings["n_components"],
        max_iter = settings["max_iter"],
        random_state = settings["random_state"]
        )
    ica.fit(filt_raw, picks=settings["ch_names"])

    # Remove ICA components
    ica.exclude(settings["ica_exclude"])
    ica.apply(filt_raw)

    # Apply LPF
    band_raw = filt_raw.copy().filter(
        l_freq=None,
        h_freq=settings["lpf_fc"],
        picks=settings["ch_names"]
        )
    



Creating RawArray with float64 data, n_channels=15, n_times=1007744
    Range : 0 ... 1007743 =      0.000 ...  3936.496 secs
Ready.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


RuntimeError: bad channel(s) ['P07'] marked do not exist in info